# Generate Experiments

In [1]:
graph_base_dir_dean="/nvmeSpace/fuchs/deans-datasets/"
graph_base_dir_others="/nvmeSpace/fuchs/graphalytics-datasets/"
graph_logs_base_dir = "/nvmeSpace/fuchs/graphlogs/"

csr_lcc="csr3-lcc"
csr="csr3"

llama="llama8-ref"
stinger="stinger7-ref"
teseo_lcc="teseo-lcc.12"
teseo_lcc_dv="teseo-lcc-dv.12b"
livegraph="livegraph3_ro"
sortledton="sortledton.1"
graphone="g1_v6-ref-ignore-build"
sortledton="sortledton"

graph500_22 = "graph500-22"
graph500_24 = "graph500-24"
graph500_26 = "graph500-26"
uniform_24 = "uniform-24"
uniform_26 = "uniform-26"
dota_legaue = "dota-league"
friendster = "com-friendster"

library_2_write_threads={
    llama: 16,
    livegraph: 20,
    stinger: 56,
    teseo_lcc: 56,
    teseo_lcc_dv: 56,
    sortledton: 56,
    graphone: 20,
    csr_lcc: 56,
    sortledton: 56,
    csr: 56
}

graph_2_graphlog = {
    graph500_24: graph_logs_base_dir + "graph500-24-1.0.graphlog",
    uniform_24: graph_logs_base_dir + "uniform24-24-1.0.graphlog"
}

blacklist = {
    llama: [graph500_26, uniform_26, friendster],
    stinger: [graph500_26, uniform_26, friendster], # TODO fix?
    graphone: [friendster] # TODO fix
}

blacklist_algo = {
    (llama, graph500_24): "lcc",
    (graphone, friendster): "lcc"
}

In [2]:
not_deans_datasets = [friendster]

def get_graph(name, dense):
    dean = name not in not_deans_datasets
    if dean:
        d = graph_base_dir_dean
    else:
        d = graph_base_dir_others + name + "/"
    if dense:
        name += "-dense"
    d += name
    d += ".properties"
    return d

def get_graph_lib(library, name):
    if library == teseo_lcc_dv:
        return get_graph(name, True)
    else:
        return get_graph(name, False)

In [3]:
all_libraries=[csr_lcc,teseo_lcc,teseo_lcc_dv,stinger,livegraph,llama,graphone,sortledton]
all_graphs=[dota_legaue, graph500_22, graph500_24, uniform_24, graph500_26, uniform_26, friendster]

libraries=[stinger,llama,graphone,livegraph,teseo_lcc]
graphs=[graph500_24,uniform_24]

writer_threads = [0]

analytics = True

block_sizes=[512]

algo_blacklist="lcc"

run_updates = True
measure_memory = True

default_parameters = {
    "-u": "",
    "-R": 5,
    "-d": "results.sqlite3"
}

if not analytics:
    default_parameters["-R"] = 0

In [4]:
import copy

experiments = []
for g in graphs:
    for l in libraries:
        for wt in writer_threads:
            if l in blacklist and g in blacklist[l]:
                continue        

            parameters = copy.copy(default_parameters)
            parameters["-l"] = l
            parameters["-G"] = get_graph_lib(l, g)
            if wt == 0:
                parameters["-w"] = library_2_write_threads[l]
            else:
                parameters["-w"] = wt

            if l in [csr_lcc, csr]:
                parameters["--load"] = ""
            
            
            if algo_blacklist:
                parameters["--blacklist"] = algo_blacklist  
            elif (l, g) in blacklist_algo:
                parameters["--blacklist"] = blacklist_algo[(l, g)]

                
            if l == llama:
                parameters["--build_frequency"] = "10s"
            
            if run_updates:
                parameters["--log"] = graph_2_graphlog[g]
                if l == llama:
                    parameters["--aging_timeout"] = "4h"
                    parameters["--aging_memfp"] = ""
                    parameters["--aging_memfp_physical"] = ""
                    parameters["--aging_release_memory"] = "false"
                    
                    if l == llama:
                        parameters["--aging_memfp_threshold"] = "230G"
            
            if l == sortledton:
                for bs in block_sizes:
                    parameters["--block_size"] = bs
                    ex = ""
                    for (k, v) in parameters.items():
                        ex += " " + k + " " + str(v)

                    experiments.append(ex)
            else:    
                ex = ""
                for (k, v) in parameters.items():
                    ex += " " + k + " " + str(v)
                    
            

                experiments.append(ex)
experiments = list(map(lambda e: "./gfe_driver " + e, experiments))
experiments = list(map(lambda e: "timeout 8h " + e, experiments))
for e in experiments:
    print(e)

timeout 8h ./gfe_driver  -u  -R 5 -d results.sqlite3 -l stinger7-ref -G /nvmeSpace/fuchs/deans-datasets/graph500-24.properties -w 56 --blacklist lcc --log /nvmeSpace/fuchs/graphlogs/graph500-24-1.0.graphlog
timeout 8h ./gfe_driver  -u  -R 5 -d results.sqlite3 -l llama8-ref -G /nvmeSpace/fuchs/deans-datasets/graph500-24.properties -w 16 --blacklist lcc --build_frequency 10s --log /nvmeSpace/fuchs/graphlogs/graph500-24-1.0.graphlog --aging_timeout 4h --aging_memfp  --aging_memfp_physical  --aging_release_memory false --aging_memfp_threshold 230G
timeout 8h ./gfe_driver  -u  -R 5 -d results.sqlite3 -l g1_v6-ref-ignore-build -G /nvmeSpace/fuchs/deans-datasets/graph500-24.properties -w 20 --blacklist lcc --log /nvmeSpace/fuchs/graphlogs/graph500-24-1.0.graphlog
timeout 8h ./gfe_driver  -u  -R 5 -d results.sqlite3 -l livegraph3_ro -G /nvmeSpace/fuchs/deans-datasets/graph500-24.properties -w 20 --blacklist lcc --log /nvmeSpace/fuchs/graphlogs/graph500-24-1.0.graphlog
timeout 8h ./gfe_driver  